In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
comparison_x_sample = pd.read_csv(
    "../output/comparison_x_sample.tsv", sep="\t", index_col=0
)

sets = kraft.gmt.read_many(kraft.path.list_("../input/set/"))

set_x_information = pd.read_csv(
    "../output/set_x_information.tsv", sep="\t", index_col=0
)

In [ ]:
for comparison in comparison_x_sample.index.to_numpy():

    directory_path = os.path.join("../output/sea", kraft.path.clean(comparison), "")

    kraft.path.path(directory_path)

    scores = pd.read_csv(
        os.path.join(
            "../output/compare_feature"",
            kraft.path.clean(comparison),
            "all_{}".format(SETTING["sea_function"]),
            "scores.tsv",
        ),
        sep="\t",
        index_col=0,
    )

    score_ = scores.loc[:, "Score"]

    score_.name = SETTING["sea_function"]

    set_scores = kraft.sea.score_samples_and_sets(
        score_.to_frame(),
        {
            set_: sets[set_]
            for set_ in set_x_information.index.to_numpy()[
                set_x_information.loc[:, "Selected"].to_numpy()
            ]
        },
        n_job=SETTING["n_job"],
    ).squeeze()

    set_scores.to_csv(
        os.path.join(directory_path, "{}.tsv".format(SETTING["sea_function"])),
        sep="\t",
    )

    set_scores.name = "Set Score"

    directory_path = os.path.join(
        directory_path, "{}_mountain/".format(SETTING["sea_function"])
    )

    kraft.path.path(directory_path)

    for set_ in SETTING["sets_to_peek"]:

        kraft.sea.score_sample_and_set(
            set_scores,
            sets[set_],
            title="{}<br>{}".format(comparison, set_),
            html_file_path=os.path.join(directory_path, "{}.html".format(set_)),
        )